In [24]:
import folium
import geopandas as gpd
from folium.plugins import HeatMap
from IPython.display import display

In [25]:
gdf = gpd.read_file('datasets\census_data.geojson')

In [26]:
gdf.head()


,OBJECTID,ED_ID,ED_ENGLISH,ED_GAEILGE,COUNTY,CONTAE,PROVINCE,CENTROID_X,CENTROID_Y,GUID_,...,T1_1AGE55_59T,T1_1AGE60_64T,T1_1AGE65_69T,T1_1AGE70_74T,T1_1AGE75_79T,T1_1AGE80_84T,T1_1AGEGE_85T,Shape__Area,Shape__Length,geometry
0,1,47033,BALLYMODAN,,CORK,Corcaigh,Munster,547718.261772674,553339.455581436,2AE196291F5013A3E055000000000001,...,266,236,200,148,93,69,69,7.387427e+07,51033.242833,"POLYGON ((-8.69709 51.73353, -8.69837 51.73341..."
1,2,97055,KILBEACON,Cill Bhéacáin,KILKENNY,Cill Chainnigh,Leinster,657681.211216703,627000.862373916,2AE19629240313A3E055000000000001,...,4,13,9,6,3,7,4,3.551668e+07,27411.591897,"POLYGON ((-7.12786 52.40372, -7.12056 52.40202..."
2,3,128014,DOCK A,An Duga A,LIMERICK,Luimneach,Munster,557059.535581243,656662.074900637,2AE1962917E613A3E055000000000001,...,93,58,35,23,15,11,8,4.975769e+05,3043.951996,"POLYGON ((-8.63056 52.66019, -8.63364 52.65667..."
3,4,47208,KILMALODA WEST,,CORK,Corcaigh,Munster,542522.122127034,547639.394685899,2AE1962921D313A3E055000000000001,...,11,8,5,5,8,5,2,3.056677e+07,34016.989343,"POLYGON ((-8.80051 51.70059, -8.80020 51.69709..."
4,5,217114,KILVEMNON,Cill Mheanmnáin,TIPPERARY,Tiobraid Árann,Munster,635998.202174985,637668.902395454,2AE196291C1F13A3E055000000000001,...,30,21,14,15,10,7,4,5.073036e+07,46792.036472,"MULTIPOLYGON (((-7.46133 52.46589, -7.45921 52..."


In [47]:
old_age_data = gdf[['ED_ENGLISH', 'COUNTY', 'PROVINCE', 'T1_1AGE65_69T', 'T1_1AGE70_74T', 
                'T1_1AGE75_79T', 'T1_1AGE80_84T', 'T1_1AGEGE_85T']]

In [48]:
display(old_age_data)

,ED_ENGLISH,COUNTY,PROVINCE,T1_1AGE65_69T,T1_1AGE70_74T,T1_1AGE75_79T,T1_1AGE80_84T,T1_1AGEGE_85T
0,BALLYMODAN,CORK,Munster,200,148,93,69,69
1,KILBEACON,KILKENNY,Leinster,9,6,3,7,4
2,DOCK A,LIMERICK,Munster,35,23,15,11,8
3,KILMALODA WEST,CORK,Munster,5,5,8,5,2
4,KILVEMNON,TIPPERARY,Munster,14,15,10,7,4
...,...,...,...,...,...,...,...,...
3404,St. PETER'S,LOUTH,Leinster,296,220,122,73,45
3405,CURRAGHMORE,KERRY,Munster,15,16,8,2,5
3406,St. JOHNSTOWN,DONEGAL,Ulster,53,39,44,15,19
3407,KILDARE,KILDARE,Leinster,350,239,141,123,105


In [50]:
old_age_data.loc[: ,'Total_Age_65+'] = old_age_data[['T1_1AGE65_69T', 'T1_1AGE70_74T', 'T1_1AGE75_79T', 'T1_1AGE80_84T', 'T1_1AGEGE_85T']].sum(axis=1)

In [51]:
# Convert to projected CRS (Irish Transverse Mercator)
gdf = gdf.to_crs(epsg=2157)

# Now, safely calculate centroids
gdf["Latitude"] = gdf.geometry.centroid.y
gdf["Longitude"] = gdf.geometry.centroid.x

# Convert back to WGS84 for mapping
gdf = gdf.to_crs(epsg=4326)


In [52]:
# Initialize the map centered around Ireland
m = folium.Map(location=[53.5, -7.7], zoom_start=7)  # Adjust center if needed

# Prepare data for the heatmap: (latitude, longitude, population)
heat_data = gdf[["Latitude", "Longitude", "Total_Age_65+"]].values.tolist()

# Add heatmap layer
HeatMap(heat_data, radius=15, blur=10, max_zoom=10).add_to(m)

# Display the map
m

KeyError: "['Total_Age_65+'] not in index"